# Analisis Sentimen (NLP with ML)

## Data Pre-Processing / Cleaning Data

In [ ]:
import pandas as pd
import re
import string

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Import Data

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia/master/dataset_tweet_sentiment_cellular_service_provider.csv')

In [ ]:
df.head()

,Id,Sentiment,Text Tweet
0,1,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...
1,2,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDE..."
2,3,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...
3,4,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...
4,5,negative,Selamat malam PENDUSTA <PROVIDER_NAME>


In [ ]:
df= df.rename(columns={'Text Tweet':'tweet_text', 'Sentiment':'label'})

In [ ]:
df

,Id,label,tweet_text
0,1,positive,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...
1,2,positive,"Saktinya balik lagi, alhamdulillah :v <PROVIDE..."
2,3,negative,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...
3,4,negative,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...
4,5,negative,Selamat malam PENDUSTA <PROVIDER_NAME>
...,...,...,...
295,296,positive,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan..."
296,297,positive,Alhamdulillah lancar pakai <PROVIDER_NAME>
297,298,positive,"Untung pakai internet <PROVIDER_NAME>, lancar,..."
298,299,positive,"di tempat ramai seperti di lokasi wisata, <PRO..."


In [ ]:
df = df.dropna()

In [ ]:
df['label'].value_counts()

negative    161
positive    139
Name: label, dtype: int64

In [ ]:
df = df[['tweet_text', 'label']]

In [ ]:
df.head()

,tweet_text,label
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative


### Cek na dan duplicate

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300 entries, 0 to 299
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  300 non-null    object
 1   label       300 non-null    object
dtypes: object(2)
memory usage: 7.0+ KB


In [ ]:
df.isna().sum()

tweet_text    0
label         0
dtype: int64

In [ ]:
df.duplicated().sum()

2

In [ ]:
df = df.drop_duplicates().reset_index(drop = True)

In [ ]:
df

,tweet_text,label
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative
...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive


### Tokenisasi

In [ ]:
df['tokenize'] = df.apply(lambda x: nltk.word_tokenize(x['tweet_text']), axis=1)

In [ ]:
df

,tweet_text,label,tokenize
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N..."
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v..."
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban..."
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin..."
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]"
...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_..."
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM..."
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,..."
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat..."


### Case folding & remove punctuation

In [ ]:
def clean_text(text):
  text = text.lower() #lowercase atau case folding
  text = re.sub('@[^\s]+', '', text) #remove username
  text = re.sub('\[.*?\]', '', text) # remove square brackets
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', text) # remove URLs
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
  text = re.sub('\w*\d\w*', '', text)
  text = re.sub('[‘’“”…]', '', text)
  text = re.sub('\n', '', text)
  return text

In [ ]:
clean1 = lambda x: clean_text(x)

df['clean1'] = pd.DataFrame(df['tweet_text'].apply(clean1))

In [ ]:
df

,tweet_text,label,tokenize,clean1
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N...",usermention boikotprovidername gunakan produk ...
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v...",saktinya balik lagi alhamdulillah v providername
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban...",selamat pagi providername bisa bantu kenapa d...
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin...",dear providername ini jaringan data lemot ban...
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]",selamat malam pendusta providername
...,...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_...",pantesan lancar ya sinyal providername yang l...
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM...",alhamdulillah lancar pakai providername
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,...",untung pakai internet providername lancar jadi...
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat...",di tempat ramai seperti di lokasi wisata provi...


### Stemming

In [ ]:
df

,tweet_text,label,tokenize,clean1
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N...",usermention boikotprovidername gunakan produk ...
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v...",saktinya balik lagi alhamdulillah v providername
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban...",selamat pagi providername bisa bantu kenapa d...
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin...",dear providername ini jaringan data lemot ban...
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]",selamat malam pendusta providername
...,...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_...",pantesan lancar ya sinyal providername yang l...
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM...",alhamdulillah lancar pakai providername
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,...",untung pakai internet providername lancar jadi...
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat...",di tempat ramai seperti di lokasi wisata provi...


In [ ]:
text = df['clean1']

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

output = [(stemmer.stem(token)) for token in text]

In [ ]:
df['clean2'] = output

In [ ]:
df

,tweet_text,label,tokenize,clean1,clean2
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N...",usermention boikotprovidername gunakan produk ...,usermention boikotprovidername guna produk ban...
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v...",saktinya balik lagi alhamdulillah v providername,sakti balik lagi alhamdulillah v providername
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban...",selamat pagi providername bisa bantu kenapa d...,selamat pagi providername bisa bantu kenapa di...
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin...",dear providername ini jaringan data lemot ban...,dear providername ini jaring data lot banget p...
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]",selamat malam pendusta providername,selamat malam dusta providername
...,...,...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_...",pantesan lancar ya sinyal providername yang l...,pantesan lancar ya sinyal providername yang la...
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM...",alhamdulillah lancar pakai providername,alhamdulillah lancar pakai providername
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,...",untung pakai internet providername lancar jadi...,untung pakai internet providername lancar jadi...
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat...",di tempat ramai seperti di lokasi wisata provi...,di tempat ramai seperti di lokasi wisata provi...


### Stopwords

In [ ]:
tambahan = ['telkom', 'providername', 'provider', 'retweet', 'rt', 'rts', 'wkwkwk']

sw = set().union(stopwords.words('indonesian'), tambahan)

In [ ]:
len(sw)

764

In [ ]:
sw

In [ ]:
df['clean3'] = pd.DataFrame(df['clean2'].apply(lambda x: ' '.join([y for y in x.split() if y not in (sw)])))

In [ ]:
df

,tweet_text,label,tokenize,clean1,clean2,clean3
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N...",usermention boikotprovidername gunakan produk ...,usermention boikotprovidername guna produk ban...,usermention boikotprovidername produk bangsa
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v...",saktinya balik lagi alhamdulillah v providername,sakti balik lagi alhamdulillah v providername,sakti alhamdulillah v
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban...",selamat pagi providername bisa bantu kenapa d...,selamat pagi providername bisa bantu kenapa di...,selamat pagi bantu kamar sinyal hilang panggil...
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin...",dear providername ini jaringan data lemot ban...,dear providername ini jaring data lot banget p...,dear jaring data lot banget h
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]",selamat malam pendusta providername,selamat malam dusta providername,selamat malam dusta
...,...,...,...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_...",pantesan lancar ya sinyal providername yang l...,pantesan lancar ya sinyal providername yang la...,pantesan lancar ya sinyal lancar lombok timur
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM...",alhamdulillah lancar pakai providername,alhamdulillah lancar pakai providername,alhamdulillah lancar pakai
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,...",untung pakai internet providername lancar jadi...,untung pakai internet providername lancar jadi...,untung pakai internet lancar ide
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat...",di tempat ramai seperti di lokasi wisata provi...,di tempat ramai seperti di lokasi wisata provi...,ramai lokasi wisata jaya


### Data Encoding

In [ ]:
# cara yang pertama yang kita gunakan
df['label_num'] = df['label'].apply(lambda score: 1 if score=='positive' else 0)

# #cara kedua menggunakan categorical data/label encoding
# df['label_num'] = df['label'].astype('category')

# df['label_num'] = df['label'].cat.codes

In [ ]:
df

,tweet_text,label,tokenize,clean1,clean2,clean3,label_num
0,<USER_MENTION> #BOIKOT_<PROVIDER_NAME> Gunakan...,positive,"[<, USER_MENTION, >, #, BOIKOT_, <, PROVIDER_N...",usermention boikotprovidername gunakan produk ...,usermention boikotprovidername guna produk ban...,usermention boikotprovidername produk bangsa,1
1,"Saktinya balik lagi, alhamdulillah :v <PROVIDE...",positive,"[Saktinya, balik, lagi, ,, alhamdulillah, :, v...",saktinya balik lagi alhamdulillah v providername,sakti balik lagi alhamdulillah v providername,sakti alhamdulillah v,1
2,Selamat pagi <PROVIDER_NAME> bisa bantu kenap...,negative,"[Selamat, pagi, <, PROVIDER_NAME, >, bisa, ban...",selamat pagi providername bisa bantu kenapa d...,selamat pagi providername bisa bantu kenapa di...,selamat pagi bantu kamar sinyal hilang panggil...,0
3,Dear <PROVIDER_NAME> akhir2 ini jaringan data ...,negative,"[Dear, <, PROVIDER_NAME, >, akhir2, ini, jarin...",dear providername ini jaringan data lemot ban...,dear providername ini jaring data lot banget p...,dear jaring data lot banget h,0
4,Selamat malam PENDUSTA <PROVIDER_NAME>,negative,"[Selamat, malam, PENDUSTA, <, PROVIDER_NAME, >]",selamat malam pendusta providername,selamat malam dusta providername,selamat malam dusta,0
...,...,...,...,...,...,...,...
293,"Pantesan lancar ya, sinyal <PROVIDER_NAME> yan...",positive,"[Pantesan, lancar, ya, ,, sinyal, <, PROVIDER_...",pantesan lancar ya sinyal providername yang l...,pantesan lancar ya sinyal providername yang la...,pantesan lancar ya sinyal lancar lombok timur,1
294,Alhamdulillah lancar pakai <PROVIDER_NAME>,positive,"[Alhamdulillah, lancar, pakai, <, PROVIDER_NAM...",alhamdulillah lancar pakai providername,alhamdulillah lancar pakai providername,alhamdulillah lancar pakai,1
295,"Untung pakai internet <PROVIDER_NAME>, lancar,...",positive,"[Untung, pakai, internet, <, PROVIDER_NAME, >,...",untung pakai internet providername lancar jadi...,untung pakai internet providername lancar jadi...,untung pakai internet lancar ide,1
296,"di tempat ramai seperti di lokasi wisata, <PRO...",positive,"[di, tempat, ramai, seperti, di, lokasi, wisat...",di tempat ramai seperti di lokasi wisata provi...,di tempat ramai seperti di lokasi wisata provi...,ramai lokasi wisata jaya,1


### finishing clean to csv

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  298 non-null    object
 1   label       298 non-null    object
 2   tokenize    298 non-null    object
 3   clean1      298 non-null    object
 4   clean2      298 non-null    object
 5   clean3      298 non-null    object
 6   label_num   298 non-null    int64 
dtypes: int64(1), object(6)
memory usage: 16.4+ KB


In [ ]:
df.isna().sum()

tweet_text    0
label         0
tokenize      0
clean1        0
clean2        0
clean3        0
label_num     0
dtype: int64

In [ ]:
df.duplicated(subset='clean3').sum()

4

In [ ]:
df = df.drop_duplicates(subset='clean3').reset_index(drop=True)

In [ ]:
df.duplicated(subset='clean3').sum()

0

In [ ]:
df = df[['clean3', 'label_num']]

In [ ]:
df.rename({'clean3': 'text', 'label_num':'label'}, axis=1, inplace=True)

In [ ]:
df

,text,label
0,usermention boikotprovidername produk bangsa,1
1,sakti alhamdulillah v,1
2,selamat pagi bantu kamar sinyal hilang panggil...,0
3,dear jaring data lot banget h,0
4,selamat malam dusta,0
...,...,...
289,pantesan lancar ya sinyal lancar lombok timur,1
290,alhamdulillah lancar pakai,1
291,untung pakai internet lancar ide,1
292,ramai lokasi wisata jaya,1


In [ ]:
df.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.to_csv('dataset_clean.csv', index=False)

## Feature Extraction

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df

,text,label
0,usermention boikotprovidername produk bangsa,1
1,sakti alhamdulillah v,1
2,selamat pagi bantu kamar sinyal hilang panggil...,0
3,dear jaring data lot banget h,0
4,selamat malam dusta,0
...,...,...
289,pantesan lancar ya sinyal lancar lombok timur,1
290,alhamdulillah lancar pakai,1
291,untung pakai internet lancar ide,1
292,ramai lokasi wisata jaya,1


In [ ]:
df['text'] = df['text'].astype(str)

In [ ]:
vec = CountVectorizer().fit(df['text'])
vec_transform = vec.transform(df['text'])

In [ ]:
print(vec_transform)

  (0, 43)	1
  (0, 80)	1
  (0, 482)	1
  (0, 652)	1
  (1, 11)	1
  (1, 520)	1
  (2, 47)	1
  (2, 122)	1
  (2, 210)	1
  (2, 267)	1
  (2, 445)	1
  (2, 455)	1
  (2, 533)	1
  (2, 556)	1
  (2, 648)	1
  (3, 42)	1
  (3, 123)	1
  (3, 126)	1
  (3, 248)	1
  (3, 342)	1
  (4, 145)	1
  (4, 356)	1
  (4, 533)	1
  (5, 171)	1
  (5, 450)	1
  :	:
  (288, 547)	1
  (288, 573)	1
  (288, 600)	1
  (289, 322)	2
  (289, 341)	1
  (289, 457)	1
  (289, 556)	1
  (289, 612)	1
  (289, 668)	1
  (290, 11)	1
  (290, 322)	1
  (290, 447)	1
  (291, 222)	1
  (291, 227)	1
  (291, 322)	1
  (291, 447)	1
  (291, 643)	1
  (292, 250)	1
  (292, 340)	1
  (292, 503)	1
  (292, 664)	1
  (293, 15)	1
  (293, 167)	1
  (293, 556)	1
  (293, 570)	1


In [ ]:
vec_transform.shape

(294, 678)

In [ ]:
print(vec_transform[0][0])

  (0, 43)	1
  (0, 80)	1
  (0, 482)	1
  (0, 652)	1


## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = vec_transform.toarray()
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## Modelling

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model1 = MultinomialNB().fit(X_train, y_train)
model2 = DecisionTreeClassifier().fit(X_train, y_train)
model3 = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
prediksi_model1 = model1.predict(X_test)
prediksi_model2 = model2.predict(X_test)
prediksi_model3 = model3.predict(X_test)

In [ ]:
print('Akurasi model 1 adalah: ', model1.score(X_test, y_test))
print('Akurasi model 2 adalah: ', model2.score(X_test, y_test))
print('Akurasi model 3 adalah: ', model3.score(X_test, y_test))

Akurasi model 1 adalah:  0.8813559322033898
Akurasi model 2 adalah:  0.7627118644067796
Akurasi model 3 adalah:  0.8813559322033898
